In [ ]:
# Install timm (DeiT models) and other deps
!pip install -q timm

# Mount Google Drive to persist checkpoints and dataset (run interactively)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Check GPU type & free memory
!nvidia-smi -L
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv


GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-b021b83d-5c76-0ca8-c9a5-eaf82b38015c)
name, memory.total [MiB], memory.free [MiB]
NVIDIA A100-SXM4-40GB, 40960 MiB, 40506 MiB


In [ ]:
!unzip /content/archive.zip -d /content/
# then set train_dir = "/content/navarasa_data/..."


Streaming output truncated to the last 5000 lines.
  inflating: /content/NAVRASA FACIAL EMOTION IMAGE DATA/training/SHANTA/Training_52310642.jpg  
  inflating: /content/NAVRASA FACIAL EMOTION IMAGE DATA/training/SHANTA/Training_5231924.jpg  
  inflating: /content/NAVRASA FACIAL EMOTION IMAGE DATA/training/SHANTA/Training_52346818.jpg  
  inflating: /content/NAVRASA FACIAL EMOTION IMAGE DATA/training/SHANTA/Training_52357264.jpg  
  inflating: /content/NAVRASA FACIAL EMOTION IMAGE DATA/training/SHANTA/Training_52366242.jpg  
  inflating: /content/NAVRASA FACIAL EMOTION IMAGE DATA/training/SHANTA/Training_5237197.jpg  
  inflating: /content/NAVRASA FACIAL EMOTION IMAGE DATA/training/SHANTA/Training_52388864.jpg  
  inflating: /content/NAVRASA FACIAL EMOTION IMAGE DATA/training/SHANTA/Training_52442111.jpg  
  inflating: /content/NAVRASA FACIAL EMOTION IMAGE DATA/training/SHANTA/Training_52453066.jpg  
  inflating: /content/NAVRASA FACIAL EMOTION IMAGE DATA/training/SHANTA/Training_524544

In [ ]:
# Colab-ready DeiT (deit_small) training script with mixed precision and Drive checkpointing.
import os, time, math
import torch, timm
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
import numpy as np

# -------------------------
# Editable settings
# -------------------------
# Put your dataset in Drive or upload to /content/navarasa_data
# Example Drive path: /content/drive/MyDrive/navarasa_data/...
train_dir = "/content/NAVRASA FACIAL EMOTION IMAGE DATA/training"
test_dir  = "/content/NAVRASA FACIAL EMOTION IMAGE DATA/testing"

# Where to save checkpoints on Drive
ckpt_dir = "/content/drive/MyDrive/navarasa_checkpoints"
os.makedirs(ckpt_dir, exist_ok=True)
checkpoint_path = os.path.join(ckpt_dir, "deit_colab_checkpoint.pth")
best_model_path = os.path.join(ckpt_dir, "deit_colab_best.pth")

# Model and hyperparams
model_name = "deit_small_patch16_224"   # deit_small is Colab-friendly
img_size = 224
batch_size = 48      # start moderate; lower to 24/16 if OOM on your GPU
num_epochs = 30
num_classes = 9
base_lr = 3e-4
weight_decay = 0.05
warmup_epochs = 2
patience = 6
num_workers = 2      # Colab: 2 is safe; increase if you want and it works
use_mixup = False
mixup_alpha = 0.2
gradient_accumulation_steps = 1  # increase if OOM needed to get effective larger batch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------
# Build model (timm DeiT small)
# -------------------------
model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
model = model.to(device)

# -------------------------
# Transforms & datasets
# -------------------------
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(img_size, scale=(0.7,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
test_dataset  = datasets.ImageFolder(test_dir,  transform=transform_test)
assert len(train_dataset) > 0 and len(test_dataset) > 0, "No data found — check paths"

# class imbalance handling
unique_classes = np.unique(train_dataset.targets)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=train_dataset.targets)
full_weights = np.ones(num_classes, dtype=float)
for idx, cls in enumerate(unique_classes):
    full_weights[int(cls)] = class_weights[idx]
class_weights_tensor = torch.tensor(full_weights, dtype=torch.float).to(device)

# Weighted sampler
class_counts = np.bincount(train_dataset.targets, minlength=num_classes)
class_counts = np.where(class_counts == 0, 1, class_counts)
weights = 1.0 / class_counts
sample_weights = [weights[t] for t in train_dataset.targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler,
                          num_workers=num_workers, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False,
                          num_workers=num_workers, pin_memory=True)

# -------------------------
# Loss, optimizer, scheduler, amp
# -------------------------
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# Separate head/backbone lr
head_params = []
backbone_params = []
for name, p in model.named_parameters():
    if not p.requires_grad:
        continue
    if any(x in name.lower() for x in ("head", "mlp_head", "classifier", "heads", "fc", "pre_logits")):
        head_params.append(p)
    else:
        backbone_params.append(p)

if len(head_params) and len(backbone_params):
    optimizer = optim.AdamW([{'params': head_params, 'lr': base_lr},
                             {'params': backbone_params, 'lr': base_lr*0.1}],
                            weight_decay=weight_decay)
else:
    optimizer = optim.AdamW(model.parameters(), lr=base_lr, weight_decay=weight_decay)

# LR warmup (simple)
total_steps = int(len(train_loader) * num_epochs / max(1, gradient_accumulation_steps))
warmup_steps = int(len(train_loader) * warmup_epochs / max(1, gradient_accumulation_steps))
def lr_lambda(step):
    if step < warmup_steps:
        return float(step) / float(max(1, warmup_steps))
    progress = float(step - warmup_steps) / float(max(1, total_steps - warmup_steps))
    return 0.5 * (1.0 + math.cos(math.pi * progress))
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda'))

# -------------------------
# Resume from checkpoint if exists
# -------------------------
start_epoch = 0
best_val = 0.0
if os.path.exists(checkpoint_path):
    ck = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(ck['model_state_dict'])
    try:
        optimizer.load_state_dict(ck['optimizer_state_dict'])
    except Exception as e:
        print("Warning: optimizer state not loaded:", e)
    start_epoch = int(ck.get('epoch', 0))
    best_val = float(ck.get('best_val', 0.0))
    print(f"Resumed from checkpoint: epoch {start_epoch}, best_val={best_val:.4f}")

# MixUp helper
def mixup_data(x, y, alpha=0.2):
    if alpha <= 0: return x, y, None, None, 1.0
    lam = np.random.beta(alpha, alpha)
    idx = torch.randperm(x.size(0)).to(device)
    mixed_x = lam * x + (1 - lam) * x[idx]
    return mixed_x, y, y[idx], lam

# -------------------------
# Training loop
# -------------------------
train_losses, val_accs = [], []
epochs_no_improve = 0
global_step = 0

for epoch in range(start_epoch, num_epochs):
    model.train()
    running_loss = 0.0
    nbatches = 0
    t0 = time.time()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    optimizer.zero_grad()

    for step, (images, labels) in enumerate(pbar):
        images, labels = images.to(device), labels.to(device)
        if use_mixup:
            mixed_x, y_a, y_b, lam = mixup_data(images, labels, mixup_alpha)
            inputs = mixed_x
        else:
            inputs = images

        with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
            outputs = model(inputs)
            if use_mixup:
                loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
            else:
                loss = criterion(outputs, labels)
            loss = loss / gradient_accumulation_steps

        scaler.scale(loss).backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            scheduler.step()

        running_loss += loss.item() * gradient_accumulation_steps
        nbatches += 1
        pbar.set_postfix({'loss': f'{running_loss/nbatches:.4f}', 'lr': optimizer.param_groups[0]['lr']})

    avg_loss = running_loss / nbatches if nbatches else 0.0
    train_losses.append(avg_loss)

    # Validation
    model.eval()
    correct = 0
    total = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Val", leave=False):
            images, labels = images.to(device), labels.to(device)
            with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
                outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = correct / total if total else 0.0
    val_accs.append(val_acc)
    print(f"Epoch {epoch+1} loss={avg_loss:.4f} val_acc={val_acc:.4f} time={time.time()-t0:.1f}s")

    # Save checkpoint to Drive
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_val': best_val
    }, checkpoint_path)

    # Best model & early stopping
    if val_acc > best_val:
        best_val = val_acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with val_acc={best_val:.4f}")
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epoch(s).")
        if epochs_no_improve >= patience:
            print("Early stopping.")
            break

# Load best and final eval
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path, map_location=device))
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
            outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Final classification report:")
from sklearn.metrics import classification_report
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))


Using device: cuda


/tmp/ipython-input-4080248599.py:125: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda'))
Epoch 1/30:   0%|          | 0/640 [00:00<?, ?it/s]/tmp/ipython-input-4080248599.py:174: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
Val:   0%|          | 0/266 [00:00<?, ?it/s]/tmp/ipython-input-4080248599.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):


Epoch 1 loss=1.2847 val_acc=0.3916 time=82.0s
New best model saved with val_acc=0.3916


Epoch 2 loss=0.9195 val_acc=0.3821 time=81.7s
No improvement for 1 epoch(s).


Epoch 3 loss=0.7883 val_acc=0.4965 time=87.2s
New best model saved with val_acc=0.4965


...

Epoch 28 loss=0.0358 val_acc=0.9540 time=87.0s
New best model saved with val_acc=0.9540


Epoch 29 loss=0.0349 val_acc=0.9542 time=82.7s
New best model saved with val_acc=0.9542


Epoch 30 loss=0.0340 val_acc=0.9543 time=83.7s
New best model saved with val_acc=0.9543


/tmp/ipython-input-4080248599.py:249: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):


Final classification report:
              precision    recall  f1-score   support

     ADBHUTA       0.90      1.00      0.94       250
   BHAYANAKA       0.98      0.97      0.98      2000
   BIBHATSYA       1.00      1.00      1.00       500
       HASYA       0.81      0.97      0.88      1000
      KARUNA       0.58      0.93      0.71        84
      RAUDRA       0.98      0.91      0.94      2000
      SHANTA       0.99      0.99      0.99      5000
   SHRINGARA       0.97      0.76      0.85      1000
       VEERA       0.87      0.97      0.92       900

    accuracy                           0.95     12734
   macro avg       0.90      0.94      0.91     12734
weighted avg       0.96      0.95      0.95     12734

